# 讀 Markdown 轉成 Pandas DataFrame

In [45]:
import re
import pandas as pd

def parse_markdown_line(line):
    # 記錄 indent
    indent_level = len(line) - len(line.lstrip())

    # 記錄 bullet
    bullet_match = re.match(r'^\s*[-*1-9]+\.?\s+', line)
    if bullet_match:
        bullet = bullet_match.group().strip()
        line = line[len(bullet):].strip()
    else:
        bullet = None

    # 記錄 tags
    tags = re.findall(r'#\w+', line)
    line = line.rstrip(''.join(tags))

    # 删除 bullets
    line = re.sub(r'^\s*[-*1-9]+\.?\s+', '', line).strip()

    return {
        'text': line,
        'bullet': bullet,
        'tags': tags,
        'indent_level': indent_level
    }

In [88]:
def locate_lists(df):
    current_list = None
    for i, row in df.iterrows():
        tags = row['tags']
        indent_level = row['indent_level']

        if 'ol' in tags or 'ul' in tags or 'listToTable' in tags:
            if current_list is None or indent_level > current_list['indent']:
                # 新的列表开始
                current_list = {
                    'type': next(tag for tag in tags if tag in ['ol', 'ul', 'listToTable']),
                    'indent': indent_level,
                    'start': i
                }
                j = i
                while j < len(df):
                    if df.at[j, 'indent_level'] <= indent_level:
                        break
                    df.at[j, 'tags'].append('#listItem')
                    j += 1
            elif indent_level < current_list['indent']:
                # 列表结束
                current_list = None
        elif current_list and indent_level <= current_list['indent']:
            # 列表结束
            current_list = None

    return df

In [89]:
with open('test_md/test.md', 'r') as f:
    lines = f.readlines()

data = [parse_markdown_line(line) for line in lines]
df = pd.DataFrame(data)

df = locate_lists(df)

In [90]:
df

,text,bullet,tags,indent_level
0,的味道去哪裡了的要也不知,-,[#h1],0
1,然就，冰淇淋的衣：露出想要的慾望好可，清的的很的前種程度，居居然有給了，不一跟我即果。次有重...,-,[],4
2,體了不下班，得好也沒的原受到之前。,-,[#h2],4
3,的點個人，他真的可以已經至，根好像默默片是拿憶⋯以在一起了就終前道真的很電⋯我陣子應該就知道...,-,[],8
4,下這全世界⋯謝謝是因為很適合,-,[#h2],4
5,加出現盼開了悲傷角結弦：,-,[#ol],8
6,的是的好，黏土的地方,1.,[],12
7,一樣是看起來大家一造的他。,2.,[],12
8,什麼時大家是會做，,3.,[],12
9,下午判斷標題，樣不給道他們有對都沒有就是，製造偷偷什麼不讓我亮把自真的很，不在意可以啊個人負...,-,[],8


# 將原始文字轉換成內文

找到 list，包括 #ul, #ol, #listToTable，保留 indents/ bullets